<a href="https://colab.research.google.com/github/phdtrong/PortfolioProjects/blob/phdtrong-patch-main-portfolio/project10/StockPricePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
#Library importing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

In [130]:
#Training Setting
split_ratio = 0.5
stock = ['TSLA']

#Fetch the data - Select date range
import yfinance as yf
data = yf.download(stock, period='max')
data.reset_index(inplace=True)

#Select filter condition
data_close_price = data['Adj Close']

[*********************100%%**********************]  1 of 1 completed


In [131]:
# Define train and test data set
split_size = (int)(split_ratio * data_close_price.size)
train_data = data_close_price[:split_size]
test_data = data_close_price[split_size:]
#type(train_data)

#Normalize data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_data = train_data.values.reshape(-1,1) #single column, multiple rows
test_data = test_data.values.reshape(-1,1) #single column, multiple rows

In [136]:
# Train the Scaler with training data and smooth data
smoothing_window_size = (int)(split_size / 4)
even_window_size = data_close_price.size - (int)(data_close_price.size % smoothing_window_size)

for di in range(0,split_size,smoothing_window_size):
    scaler.fit(train_data[di:di+smoothing_window_size,:])
    train_data[di:di+smoothing_window_size,:] = scaler.transform(train_data[di:di+smoothing_window_size,:])

# You normalize the last bit of remaining data
if(data_close_price.size % smoothing_window_size > 0):
  scaler.fit(train_data[di+smoothing_window_size:,:])
  train_data[di+smoothing_window_size:,:] = scaler.transform(train_data[di+smoothing_window_size:,:])

IndexError: ignored

In [139]:
# Reshape both train and test data
train_data = train_data.reshape(-1)

# Normalize test data
test_data = test_data.reshape(-1)

In [135]:
# Now perform exponential moving average smoothing
# So the data will have a smoother curve than the original ragged data
EMA = 0.0
gamma = 0.1
for ti in range(split_size):
  EMA = gamma*train_data[ti] + (1-gamma)*EMA
  train_data[ti] = EMA

# Used for visualization and test purposes
all_mid_data = np.concatenate([train_data,test_data],axis=0)

In [ ]:
window_size = 100
N = train_data.size
std_avg_predictions = []
std_avg_x = []
mse_errors = []

for pred_idx in range(window_size,N):

    if pred_idx >= N:
        date = dt.datetime.strptime(k, '%Y-%m-%d').date() + dt.timedelta(days=1)
    else:
        date = data.loc[pred_idx,'Date']

    std_avg_predictions.append(np.mean(train_data[pred_idx-window_size:pred_idx]))
    mse_errors.append((std_avg_predictions[-1]-train_data[pred_idx])**2)
    std_avg_x.append(date)

print('MSE error for standard averaging: %.5f'%(0.5*np.mean(mse_errors)))